In [7]:
# ============================================================
# Collocations (PMI, t-score) around target words (no lemmatization)
# Input: a ZIP with .txt files (each contains a 'text:' or 'text::' section)
# Output: CSVs with top collocates and a ZIP to download
# ============================================================

import re, os, io, zipfile, math, unicodedata
from collections import Counter
import pandas as pd
from google.colab import files
from IPython.display import display

# ---- 1) Upload ZIP ----
print("⬆️ Choose a ZIP containing .txt leaflets (each file has a 'text:' section).")
uploaded = files.upload()
zip_name = list(uploaded.keys())[0]

extract_dir = "/content/leaflets"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(io.BytesIO(uploaded[zip_name]), 'r') as z:
    z.extractall(extract_dir)
print(f"Extracted to: {extract_dir}")

# ---- 2) Parameters ----
WINDOW = 5           # collocation window ±5 tokens
MIN_COOC = 3         # minimum observed co-occurrences to keep a pair
MIN_FREQ_W = 5       # minimum corpus frequency of the neighbor word
MIN_TOKEN_LEN = 2    # drop 1-char tokens (noise)

# Targets: list of surface forms (lowercased)
TARGETS = {
    "fasisms": {
        "forms": {
            "fašisms","fašistu","fašistam","fašistiem","fašistiskā","fašistisko","fašistiskās",
            "fašistiska","fašistiski","fašistiskie","fašistiskos","fašistisks","fašistiskums",
            "fašismu","fašisma","fašismā"
        }
    },
    "Ulmanis": {
        "forms": {
            "ulmanis","ulmani","ulmaņa","ulmanim","ulmanī","ulmanītis","ulmanītim","ulmanīti"
        }
    }
}

# Minimal Latvian stopword set (extend as needed)
STOPWORDS = set("""
un bet vai ka kā par ar no uz pie lai tas tā šis šie šīs šo šajā tad arī gan vien
tiek tika būs būtu bijis biju jūs mēs viņi viņš viņa tās tie to tam tajā tās
es man mani manis mums jums viņiem viņam viņas viņu tās tās
nav ir bija bijām bijāt esam esat būt
""".split())

# Tokenization: Latvian letters + digits (keeps diacritics)
TOKEN_RE = re.compile(r"[0-9A-Za-zĀĒĪŪŌĢĶĻŅČŖŠŪŽāēīūōģķļņčŗšūžĻŅĀĒĪŪČĢĶŠŽŖ]+", re.UNICODE)

def read_leaflet_text(path: str) -> str:
    """Return the text after the 'text:' label; fallback to whole file if missing."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()
    m = re.search(r"\btext\s*:\s*", content, flags=re.IGNORECASE)
    return content[m.end():].strip() if m else content

def normalize_text(txt: str) -> str:
    """Unicode-normalize and lowercase."""
    txt = unicodedata.normalize('NFC', txt)
    return txt.lower()

def tokenize(text: str):
    """Simple regex tokenization; keep Latvian diacritics."""
    toks = TOKEN_RE.findall(text)
    # Drop tokens with digits and length < MIN_TOKEN_LEN (reduce noise like 'b', '1934')
    toks = [t for t in toks if t.isalpha() and len(t) >= MIN_TOKEN_LEN]
    return toks

# ---- 3) Read and tokenize all documents ----
docs_tokens = []
for root, _, files_in in os.walk(extract_dir):
    for fn in files_in:
        if fn.lower().endswith(".txt"):
            p = os.path.join(root, fn)
            t = read_leaflet_text(p)
            if t.strip():
                toks = tokenize(normalize_text(t))
                if toks:
                    docs_tokens.append(toks)

print(f"Documents found: {len(docs_tokens)}")

# Flat frequency over the whole corpus
flat = [w for doc in docs_tokens for w in doc]
N_TOKENS = len(flat)
freq_w = Counter(flat)
print(f"Tokens: {N_TOKENS:,}, unique types: {len(freq_w):,}")

# ---- 4) Collocations for each target ----
def collocations_for_forms(target_forms, docs_tokens, window=5):
    """Count co-occurrences within a ±window around target forms."""
    cooc = Counter()
    f_target = 0
    tset = set(target_forms)
    for toks in docs_tokens:
        L = len(toks)
        for i, tok in enumerate(toks):
            if tok in tset:
                f_target += 1
                left = max(0, i - window)
                right = min(L, i + window + 1)
                for j in range(left, right):
                    if j == i:
                        continue
                    w = toks[j]
                    if w in tset or w in STOPWORDS or not TOKEN_RE.fullmatch(w):
                        continue
                    # skip tokens with digits or too short (double-check for robustness)
                    if not w.isalpha() or len(w) < MIN_TOKEN_LEN:
                        continue
                    cooc[w] += 1
    return cooc, f_target

def compute_stats_table(cooc, f_target, N_tokens, freq_w, window=5,
                        min_cooc=3, min_freq_w=5):
    """
    Compute expected counts (independence assumption), t-score and PMI (log2).
    E ≈ f_w * f_target * (2*window) / N_tokens
    t-score = (O - E) / sqrt(O)
    PMI = log2( O * N_tokens / (f_w * f_target * 2*window) )
    """
    rows = []
    span = 2 * window
    for w, O in cooc.items():
        f_w = freq_w.get(w, 0)
        if O < min_cooc or f_w < min_freq_w:
            continue
        E = (f_w * f_target * span) / max(N_tokens, 1)
        t_score = (O - E) / (O ** 0.5) if O > 0 else 0.0
        denom = (f_w * f_target * span)
        pmi = math.log2((O * N_tokens) / denom) if denom > 0 and O > 0 else float('-inf')
        rows.append((w, O, E, t_score, pmi, f_w))
    df = pd.DataFrame(rows, columns=["word","O_obs","E_exp","t_score","PMI","freq_w"])
    df_t = df.sort_values(["t_score","O_obs"], ascending=[False, False])
    df_p = df.sort_values(["PMI","O_obs"], ascending=[False, False])
    return df, df_t, df_p

results = {}
for tgt_name, cfg in TARGETS.items():
    cooc, f_tgt = collocations_for_forms(cfg["forms"], docs_tokens, window=WINDOW)
    df, df_t, df_pmi = compute_stats_table(cooc, f_tgt, N_TOKENS, freq_w,
                                           window=WINDOW, min_cooc=MIN_COOC, min_freq_w=MIN_FREQ_W)
    results[tgt_name] = {"raw": df, "by_t": df_t, "by_pmi": df_pmi, "f_target": f_tgt}
    print(f"\n{tgt_name}: target occurrences = {f_tgt}, kept candidates = {len(df)}")

# ---- 5) Save and preview ----
out_dir = "/content/collocations_out"
os.makedirs(out_dir, exist_ok=True)

for name, tabs in results.items():
    tabs["raw"].to_csv(f"{out_dir}/collocations_{name}_raw.csv", index=False)
    tabs["by_t"].head(200).to_csv(f"{out_dir}/collocations_{name}_top_tscore.csv", index=False)
    tabs["by_pmi"].head(200).to_csv(f"{out_dir}/collocations_{name}_top_pmi.csv", index=False)

print(f"\nCSVs saved to {out_dir}")

for name, tabs in results.items():
    print(f"\n=== {name}: TOP-50 by t-score (window ±{WINDOW}) ===")
    display(tabs["by_t"].head(50).reset_index(drop=True))
    print(f"\n=== {name}: TOP-50 by PMI (window ±{WINDOW}) ===")
    display(tabs["by_pmi"].head(50).reset_index(drop=True))

# Bundle results into a ZIP for download
out_zip = "/content/collocations_results.zip"
with zipfile.ZipFile(out_zip, 'w', compression=zipfile.ZIP_DEFLATED) as z:
    for fn in os.listdir(out_dir):
        z.write(os.path.join(out_dir, fn), arcname=fn)

print("\n⬇️ Download the results ZIP below.")
files.download(out_zip)


⬆️ Choose a ZIP containing .txt leaflets (each file has a 'text:' section).


Saving 1934-1935_partly_1936_leaflets_with_texts.zip to 1934-1935_partly_1936_leaflets_with_texts.zip
Extracted to: /content/leaflets
Documents found: 386
Tokens: 223,652, unique types: 17,388

fasisms: target occurrences = 2814, kept candidates = 1453

Ulmanis: target occurrences = 1112, kept candidates = 603

CSVs saved to /content/collocations_out

=== fasisms: TOP-50 by t-score (window ±5) ===


,word,O_obs,E_exp,t_score,PMI,freq_w
0,pret,862,323.106970,18.354769,1.415676,2568
1,ulmaņa,396,92.352226,15.258875,2.100282,734
2,nost,278,67.691413,12.613471,2.038040,538
3,valdība,212,59.638903,10.464203,1.829739,474
4,karu,190,52.089675,10.005067,1.866930,414
5,diktatūru,138,27.428863,9.412440,2.330902,218
6,valdību,126,23.905890,9.095266,2.397986,190
7,diktatūras,76,12.330406,7.303403,2.623779,98
8,valdības,96,29.945272,6.741682,1.680706,238
9,teroru,70,20.131275,5.960453,1.797916,160



=== fasisms: TOP-50 by PMI (window ±5) ===


,word,O_obs,E_exp,t_score,PMI,freq_w
0,apmātais,8,0.754923,2.561522,3.405599,6
1,uzbrūkošo,16,1.761487,3.559628,3.183207,14
2,komisāriem,12,1.509846,3.028247,2.990561,12
3,despotiju,10,1.258205,2.764398,2.990561,10
4,nodibināšanās,10,1.258205,2.764398,2.990561,10
5,ungārijas,10,1.258205,2.764398,2.990561,10
6,aģentūru,10,1.258205,2.764398,2.990561,10
7,krīt,8,1.006564,2.472553,2.990561,8
8,atmaskosim,8,1.006564,2.472553,2.990561,8
9,atsistu,6,0.754923,2.141294,2.990561,6


...


=== Ulmanis: TOP-50 by PMI (window ±5) ===


,word,O_obs,E_exp,t_score,PMI,freq_w
0,aģentūru,10,0.497201,3.005049,4.330027,10
1,apspiež,8,0.397761,2.687797,4.330027,8
2,iedama,6,0.298321,2.327701,4.330027,6
3,sašauto,6,0.298321,2.327701,4.330027,6
4,baloža,40,2.088244,5.994375,4.259638,42
5,tēvs,6,0.397761,2.287105,3.914989,8
6,zaglis,6,0.397761,2.287105,3.914989,8
7,saziņā,6,0.397761,2.287105,3.914989,8
8,celmiņu,6,0.397761,2.287105,3.914989,8
9,rokaspuiši,14,0.994402,3.475892,3.815454,20



⬇️ Download the results ZIP below.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>